In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from arch import arch_model
from joblib import Parallel, delayed
import multiprocessing
import math
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

/Users/azrarahman/github projects/garch/venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
daily = '/Users/azrarahman/github projects/garch/daily.csv'

daily_df = pd.read_csv(daily)

daily_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Unnamed: 7
0,9/17/2014,116.466003,117.043503,113.105499,114.333504,114.333504,21056800.0,NaN
1,9/18/2014,114.214996,114.214996,103.276001,106.110001,106.110001,34483200.0,NaN
2,9/19/2014,106.025749,106.958748,96.133003,98.698998,98.698998,37919700.0,NaN
3,9/20/2014,98.668251,105.823998,97.470749,102.225998,102.225998,36863600.0,NaN
4,9/21/2014,102.021248,103.106499,98.295250,99.705254,99.705254,26580100.0,NaN


In [4]:
daily_df = daily_df.drop('Unnamed: 7', axis=1)

daily_df['Date'] = pd.to_datetime(daily_df['Date'])

daily_df = daily_df.set_index('Date')
daily_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,116.466003,117.043503,113.105499,114.333504,114.333504,21056800.0
2014-09-18,114.214996,114.214996,103.276001,106.110001,106.110001,34483200.0
2014-09-19,106.025749,106.958748,96.133003,98.698998,98.698998,37919700.0
2014-09-20,98.668251,105.823998,97.470749,102.225998,102.225998,36863600.0
2014-09-21,102.021248,103.106499,98.295250,99.705254,99.705254,26580100.0


In [5]:
daily_df['log_ret'] = np.log(daily_df['Adj Close']).diff()

daily_df['variance'] = daily_df['log_ret'].rolling(180).var()

daily_df = daily_df['2020':]

def predict_volatility(x):
    
    best_model = arch_model(y=x,
                            p=1,
                            q=3).fit(update_freq=5,
                                     disp='off')
    
    variance_forecast = best_model.forecast(horizon=1).variance.iloc[-1,0]

    print(x.index[-1])
    
    return variance_forecast

daily_df['predictions'] = daily_df['log_ret'].rolling(180).apply(lambda x: predict_volatility(x))

daily_df = daily_df.dropna()

daily_df

/Users/azrarahman/github projects/garch/venv/lib/python3.11/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002459. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/Users/azrarahman/github projects/garch/venv/lib/python3.11/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002459. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resi

2020-06-28 00:00:00
2020-06-29 00:00:00
2020-06-30 00:00:00
2020-07-01 00:00:00
2020-07-02 00:00:00
2020-07-03 00:00:00
2020-07-04 00:00:00
2020-07-05 00:00:00
2020-07-06 00:00:00
2020-07-07 00:00:00
2020-07-08 00:00:00
2020-07-09 00:00:00
2020-07-10 00:00:00
2020-07-11 00:00:00
2020-07-12 00:00:00
2020-07-13 00:00:00
2020-07-14 00:00:00
2020-07-15 00:00:00
2020-07-16 00:00:00
2020-07-17 00:00:00
2020-07-18 00:00:00
2020-07-19 00:00:00
2020-07-20 00:00:00
2020-07-21 00:00:00
2020-07-22 00:00:00
2020-07-23 00:00:00
2020-07-24 00:00:00
2020-07-25 00:00:00
2020-07-26 00:00:00
2020-07-27 00:00:00
2020-07-28 00:00:00
2020-07-29 00:00:00
2020-07-30 00:00:00
2020-07-31 00:00:00
2020-08-01 00:00:00
2020-08-02 00:00:00
2020-08-03 00:00:00
2020-08-04 00:00:00
2020-08-05 00:00:00
2020-08-06 00:00:00
2020-08-07 00:00:00
2020-08-08 00:00:00
2020-08-09 00:00:00
2020-08-10 00:00:00
2020-08-11 00:00:00
2020-08-12 00:00:00
2020-08-13 00:00:00
2020-08-14 00:00:00
2020-08-15 00:00:00
2020-08-16 00:00:00


/Users/azrarahman/github projects/garch/venv/lib/python3.11/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002369. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/Users/azrarahman/github projects/garch/venv/lib/python3.11/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002369. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resi

2020-08-20 00:00:00
2020-08-21 00:00:00
2020-08-22 00:00:00
2020-08-23 00:00:00
2020-08-24 00:00:00
2020-08-25 00:00:00
2020-08-26 00:00:00
2020-08-27 00:00:00
2020-08-28 00:00:00
2020-08-29 00:00:00
2020-08-30 00:00:00
2020-08-31 00:00:00
2020-09-01 00:00:00
2020-09-02 00:00:00
2020-09-03 00:00:00
2020-09-04 00:00:00
2020-09-05 00:00:00
2020-09-06 00:00:00
2020-09-07 00:00:00
2020-09-08 00:00:00
2020-09-09 00:00:00
2020-09-10 00:00:00
2020-09-11 00:00:00
2020-09-12 00:00:00
2020-09-13 00:00:00
2020-09-14 00:00:00


/Users/azrarahman/github projects/garch/venv/lib/python3.11/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002405. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/Users/azrarahman/github projects/garch/venv/lib/python3.11/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002401. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resi

...

2023-07-19 00:00:00
2023-07-20 00:00:00
2023-07-21 00:00:00
2023-07-22 00:00:00
2023-07-23 00:00:00
2023-07-24 00:00:00
2023-07-25 00:00:00
2023-07-26 00:00:00
2023-07-27 00:00:00
2023-07-28 00:00:00
2023-07-29 00:00:00
2023-07-30 00:00:00
2023-07-31 00:00:00
2023-08-01 00:00:00
2023-08-02 00:00:00
2023-08-03 00:00:00
2023-08-04 00:00:00
2023-08-05 00:00:00
2023-08-06 00:00:00
2023-08-07 00:00:00
2023-08-08 00:00:00
2023-08-09 00:00:00
2023-08-10 00:00:00
2023-08-11 00:00:00
2023-08-12 00:00:00


/Users/azrarahman/github projects/garch/venv/lib/python3.11/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0005807. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/Users/azrarahman/github projects/garch/venv/lib/python3.11/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0005806. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(

2023-08-13 00:00:00
2023-08-14 00:00:00
2023-08-15 00:00:00
2023-08-16 00:00:00
2023-08-17 00:00:00
2023-08-18 00:00:00
2023-08-19 00:00:00
2023-08-20 00:00:00
2023-08-21 00:00:00
2023-08-22 00:00:00
2023-08-23 00:00:00
2023-08-24 00:00:00
2023-08-25 00:00:00
2023-08-26 00:00:00
2023-08-27 00:00:00
2023-08-28 00:00:00
2023-08-29 00:00:00
2023-08-30 00:00:00
2023-08-31 00:00:00
2023-09-01 00:00:00
2023-09-02 00:00:00
2023-09-03 00:00:00
2023-09-04 00:00:00
2023-09-05 00:00:00
2023-09-06 00:00:00
2023-09-07 00:00:00
2023-09-08 00:00:00
2023-09-09 00:00:00
2023-09-10 00:00:00
2023-09-11 00:00:00
2023-09-12 00:00:00
2023-09-13 00:00:00
2023-09-14 00:00:00
2023-09-15 00:00:00
2023-09-16 00:00:00
2023-09-17 00:00:00
2023-09-18 00:00:00


/Users/azrarahman/github projects/garch/venv/lib/python3.11/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0005152. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/Users/azrarahman/github projects/garch/venv/lib/python3.11/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0005151. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(

,Open,High,Low,Close,Adj Close,Volume,log_ret,variance,predictions
Date,,,,,,,,,
2020-06-28,2262.115234,2299.386719,2243.881348,2285.895508,2285.895508,1.456087e+10,0.010797,0.002473,0.000728
2020-06-29,2285.007324,2309.393311,2260.468994,2297.713623,2297.713623,1.646055e+10,0.005157,0.002473,0.000651
2020-06-30,2296.395264,2304.458984,2271.209473,2284.498291,2284.498291,1.573580e+10,-0.005768,0.002468,0.000877
2020-07-01,2286.496338,2327.438721,2276.183838,2307.081299,2307.081299,1.597155e+10,0.009837,0.002455,0.000618
2020-07-02,2307.784912,2318.740723,2259.155762,2280.852539,2280.852539,1.633892e+10,-0.011434,0.002456,0.000703
...,...,...,...,...,...,...,...,...,...
2023-09-14,6557.069336,6693.655762,6542.862793,6634.918457,6634.918457,1.381136e+10,0.011801,0.000393,0.000334
2023-09-15,6633.454590,6710.124512,6560.175293,6652.173340,6652.173340,1.147974e+10,0.002597,0.000385,0.000292
2023-09-16,6651.549805,6688.692383,6618.472656,6642.070313,6642.070313,7.402031e+09,-0.001520,0.000384,0.000286


We'll stop here. The original GARCH model code creates the columns: log ret, vol history, vol pred, pred prem, prem std and signal.

Obviously we want to predict the log returns for the next day using the LSTM to better the signal. It's a GARCH and LSTM hybrid for the final signal. 


We create a sequence of window_size x as a changeable parameter to predict the next log_ret as a function.


In [ ]:
window_size = 10
leg_returns = df['log_ret'].dropna(values) # we ...

X, y = [], [] 





build LSRTM

3


combined signal where both models must agree


In [ ]:
df["signal_daily"] = np.where(
    (df["signal_garch"] == df["signal_lstm"]),
    df["signal_garch"],
    0
)


we have a separate lstm file. in the generate signal function, only when both garch and lstm signals are the same, trade. or do nothing.

my question is, how am i meant to evaluate my lstm?


direction_acc = (np.sign(y_pred) == np.sign(y_test)).mean()
print("Direction accuracy:", direction_acc)


2️⃣ Use MSE on test data

This answers:

“How wrong is my prediction, on average?”

Lower = better.

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)


If you have this, you can defend the model academically.

3️⃣ Compare against a dumb baseline (critical)

Create a baseline:

“Tomorrow’s return = today’s return”

baseline_pred = y_test.shift(1).dropna()
mse_baseline = mean_squared_error(
    y_test.iloc[1:], baseline_pred
)

print("Baseline MSE:", mse_baseline)


✅ Your LSTM is useful only if:

LSTM MSE < Baseline MSE


If not → it learned noise.

“The LSTM was evaluated on a hold-out test set using mean squared error and directional accuracy. Performance was compared against a naïve random-walk baseline. The model achieved lower prediction error and higher directional accuracy than the baseline, indicating it learned non-trivial temporal structure in returns.”

That’s exactly MSc-level wording.

LATER::


after this all, can use a transformer instead of an LSTM, and use black scholes. 

If you're using LSTM for stock prediction, switching to a transformer could give you a few advantages:

Long-Term Dependencies: Transformers can capture long-term relationships in data more effectively than LSTMs, which can struggle with very long sequences due to vanishing gradients.

Parallel Processing: Unlike LSTMs, which process data step-by-step, transformers can process all data at once, speeding up training and inference, especially with large datasets.

Attention Mechanism: Transformers use attention to focus on the most relevant parts of the input sequence, which can help the model identify important events or patterns in stock data that are more impactful.

Scalability: Transformers tend to scale better with larger datasets, so as you collect more stock data, the transformer might improve more easily compared to LSTMs.

However, LSTM might still be fine for your case depending on the data and the problem. Transformers can require more computational resources and may need careful tuning to perform well in time series tasks like stock prediction.



--

You're right! LSTMs (Long Short-Term Memory networks) were specifically designed to combat the vanishing gradient problem that occurs in traditional RNNs. They use special gating mechanisms to better retain long-term dependencies, which is why they're better at handling longer sequences than regular RNNs.

However, while LSTMs are much better than RNNs at retaining information over time, they can still struggle with very long sequences or data with complex patterns, like stock prices, especially when the sequence length becomes very large. This is because LSTMs still process data sequentially, and as the sequence grows, training can become slower and more challenging.

Transformers, on the other hand, use a mechanism called self-attention, which allows them to look at all parts of the input at once. This means they can capture long-range dependencies more effectively and more efficiently, especially in very large datasets.

So, even though LSTMs don’t have the same vanishing gradient issue as RNNs, transformers can still outperform them in tasks involving long-term dependencies and complex patterns.